<a href="https://colab.research.google.com/github/Chohyungun/Chohyungun/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
!pip install catboost

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 불러오기

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Daycon/2회차/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Daycon/2회차/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Daycon/2회차/sample_submission.csv')

# Feature Engineering

### 파생변수 생성
- 송하인,물품_카테고리,물품_카테고리, (송하인,물품_카테고리), (송하인,물품_카테고리) 으로 각각 groupby
- mean,max,sum 값을 새로운 파생변수로 사용  


In [ ]:
train.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [ ]:
test.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [ ]:
train_dict = {
    '운송장_건수':['mean','max','sum']
}

train_group = train.groupby('송하인_격자공간고유번호')
train_agg = train_group.agg(train_dict)

train_agg.columns=['송하인_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on='송하인_격자공간고유번호', how='left')
test= test.merge(train_agg, on='송하인_격자공간고유번호', how='left')

In [ ]:
train_dict = {
    '운송장_건수':['mean','max','sum']
}

train_group = train.groupby('수하인_격자공간고유번호')
train_agg = train_group.agg(train_dict)

train_agg.columns=['수하인_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on='수하인_격자공간고유번호', how='left')
test= test.merge(train_agg, on='수하인_격자공간고유번호', how='left')

In [ ]:
train_dict = {
    '운송장_건수':['mean','max','sum']
}

train_group = train.groupby('물품_카테고리')
train_agg = train_group.agg(train_dict)

train_agg.columns=['물품_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on='물품_카테고리', how='left')
test= test.merge(train_agg, on='물품_카테고리', how='left')

In [ ]:
train_dict = {
    '운송장_건수':['mean','max','sum']
}

train_group = train.groupby(['송하인_격자공간고유번호', '수하인_격자공간고유번호'])
train_agg = train_group.agg(train_dict)

train_agg.columns=['송하인_+'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on=['송하인_격자공간고유번호','수하인_격자공간고유번호'], how='left')
test= test.merge(train_agg, on=['송하인_격자공간고유번호','수하인_격자공간고유번호'], how='left')

In [ ]:
train_dict = {
    '운송장_건수':['mean','max','sum']
}

train_group = train.groupby(['송하인_격자공간고유번호', '물품_카테고리'])
train_agg = train_group.agg(train_dict)

train_agg.columns=['송하인_'+"_".join(x).upper() for x in train_agg.columns.ravel()]
train= train.merge(train_agg, on=['송하인_격자공간고유번호','물품_카테고리'], how='left')
test= test.merge(train_agg, on=['송하인_격자공간고유번호','물품_카테고리'], how='left')

### 결측치 확인

groupby를 train데이터를 기준으로 했기 때문에  
test에 merge했을 때 train에는 없는 새로운 송하인_격자공간고유번호가 있기 때문에 결측치가 생김

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 31683
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              31684 non-null  int64  
 1   송하인_격자공간고유번호       31684 non-null  int64  
 2   수하인_격자공간고유번호       31684 non-null  int64  
 3   물품_카테고리            31684 non-null  object 
 4   운송장_건수             31684 non-null  int64  
 5   송하인_운송장_건수_MEAN_x  31684 non-null  float64
 6   송하인_운송장_건수_MAX_x   31684 non-null  int64  
 7   송하인_운송장_건수_SUM_x   31684 non-null  int64  
 8   수하인_운송장_건수_MEAN    31684 non-null  float64
 9   수하인_운송장_건수_MAX     31684 non-null  int64  
 10  수하인_운송장_건수_SUM     31684 non-null  int64  
 11  물품_운송장_건수_MEAN     31684 non-null  float64
 12  물품_운송장_건수_MAX      31684 non-null  int64  
 13  물품_운송장_건수_SUM      31684 non-null  int64  
 14  송하인_+운송장_건수_MEAN   31684 non-null  float64
 15  송하인_+운송장_건수_MAX    31684 non-null  int64  
 16  송하인_+운송장_건수_SUM    316

In [ ]:
train.isnull().sum()

index                0
송하인_격자공간고유번호         0
수하인_격자공간고유번호         0
물품_카테고리              0
운송장_건수               0
송하인_운송장_건수_MEAN_x    0
송하인_운송장_건수_MAX_x     0
송하인_운송장_건수_SUM_x     0
수하인_운송장_건수_MEAN      0
수하인_운송장_건수_MAX       0
수하인_운송장_건수_SUM       0
물품_운송장_건수_MEAN       0
물품_운송장_건수_MAX        0
물품_운송장_건수_SUM        0
송하인_+운송장_건수_MEAN     0
송하인_+운송장_건수_MAX      0
송하인_+운송장_건수_SUM      0
송하인_운송장_건수_MEAN_y    0
송하인_운송장_건수_MAX_y     0
송하인_운송장_건수_SUM_y     0
dtype: int64

In [ ]:
test.isnull().sum()

index                   0
송하인_격자공간고유번호            0
수하인_격자공간고유번호            0
물품_카테고리                 0
송하인_운송장_건수_MEAN_x     609
송하인_운송장_건수_MAX_x      609
송하인_운송장_건수_SUM_x      609
수하인_운송장_건수_MEAN      5947
수하인_운송장_건수_MAX       5947
수하인_운송장_건수_SUM       5947
물품_운송장_건수_MEAN          0
물품_운송장_건수_MAX           0
물품_운송장_건수_SUM           0
송하인_+운송장_건수_MEAN     7714
송하인_+운송장_건수_MAX      7714
송하인_+운송장_건수_SUM      7714
송하인_운송장_건수_MEAN_y     860
송하인_운송장_건수_MAX_y      860
송하인_운송장_건수_SUM_y      860
dtype: int64

### 결측치 처리

In [ ]:
test['송하인_운송장_건수_MEAN_x'] = test['송하인_운송장_건수_MEAN_x'].fillna(train['송하인_운송장_건수_MEAN_x'].mean())
test['송하인_운송장_건수_MAX_x'] = test['송하인_운송장_건수_MAX_x'].fillna(train['송하인_운송장_건수_MAX_x'].mean())
test['송하인_운송장_건수_SUM_x'] = test['송하인_운송장_건수_SUM_x'].fillna(train['송하인_운송장_건수_SUM_x'].mean())

In [ ]:
test['송하인_운송장_건수_MEAN_y'] = test['송하인_운송장_건수_MEAN_y'].fillna(train['송하인_운송장_건수_MEAN_y'].mean())
test['송하인_운송장_건수_MAX_y'] = test['송하인_운송장_건수_MAX_y'].fillna(train['송하인_운송장_건수_MAX_y'].mean())
test['송하인_운송장_건수_SUM_y'] = test['송하인_운송장_건수_SUM_y'].fillna(train['송하인_운송장_건수_SUM_y'].mean())

In [ ]:
test['송하인_+운송장_건수_MEAN'] = test['송하인_+운송장_건수_MEAN'].fillna(train['송하인_+운송장_건수_MEAN'].mean())
test['송하인_+운송장_건수_MAX'] = test['송하인_+운송장_건수_MAX'].fillna(train['송하인_+운송장_건수_MAX'].mean())
test['송하인_+운송장_건수_SUM'] = test['송하인_+운송장_건수_SUM'].fillna(train['송하인_+운송장_건수_SUM'].mean())

In [ ]:
data= pd.concat([train,test])

In [ ]:
data

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_운송장_건수_MEAN_x,송하인_운송장_건수_MAX_x,송하인_운송장_건수_SUM_x,수하인_운송장_건수_MEAN,수하인_운송장_건수_MAX,수하인_운송장_건수_SUM,물품_운송장_건수_MEAN,물품_운송장_건수_MAX,물품_운송장_건수_SUM,송하인_+운송장_건수_MEAN,송하인_+운송장_건수_MAX,송하인_+운송장_건수_SUM,송하인_운송장_건수_MEAN_y,송하인_운송장_건수_MAX_y,송하인_운송장_건수_SUM_y
0,0,5011000595017300,2871000192069300,음반,3.0,3.738095,15.000000,1727.000000,3.000000,3.0,3.0,4.092014,48,2357,3.000000,3.00000,3.000000,3.679287,12.000000,1652.000000
1,1,4148000690043300,5011000264024400,문화컨텐츠,3.0,3.970297,15.000000,401.000000,3.000000,3.0,3.0,4.976169,118,5429,3.000000,3.00000,3.000000,3.868687,12.000000,383.000000
2,2,5011000078068400,1120000007005400,농산물,3.0,3.707110,82.000000,14391.000000,3.000000,3.0,3.0,4.322081,179,87829,3.000000,3.00000,3.000000,3.707141,82.000000,14380.000000
3,3,4127100048006400,5011000587019400,기타식품,7.0,4.448276,11.000000,129.000000,5.500000,9.0,22.0,6.088825,108,4250,7.000000,7.00000,7.000000,5.052632,11.000000,96.000000
4,4,5011000078068400,2823700010076300,농산물,3.0,3.707110,82.000000,14391.000000,3.000000,3.0,3.0,4.322081,179,87829,3.000000,3.00000,3.000000,3.707141,82.000000,14380.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7915,5011000266051200,4623000417038100,농산물,NaN,4.757576,17.000000,157.000000,NaN,NaN,NaN,4.322081,179,87829,4.879308,4.96342,5.137293,4.757576,17.000000,157.000000
7916,7916,1154500001098300,5011000264055100,문화컨텐츠,NaN,4.879308,32.239616,2885.648435,10.000000,10.0,10.0,4.976169,118,5429,4.879308,4.96342,5.137293,4.879308,29.749905,2809.058989
7917,7917,5013000610049100,1147000018091400,농산물,NaN,3.597207,31.000000,6439.000000,NaN,NaN,NaN,4.322081,179,87829,4.879308,4.96342,5.137293,3.598319,31.000000,6423.000000
7918,7918,5013000610049100,3117000039026100,농산물,NaN,3.597207,31.000000,6439.000000,NaN,NaN,NaN,4.322081,179,87829,4.879308,4.96342,5.137293,3.598319,31.000000,6423.000000


In [ ]:
data['송하인_']= data['송하인_격자공간고유번호'].astype('str')+'-'+data['수하인_격자공간고유번호'].astype('str')

### Label Encoding

In [ ]:
object_columns=['송하인_격자공간고유번호','수하인_격자공간고유번호','물품_카테고리','물품_카테고리','송하인_']
for column in object_columns:
    data[column] = pd.factorize(data[column])[0]

In [ ]:
data=data.drop('index',axis=1) # index column 제거

In [ ]:
# 다시 train, test로 분리
train = data[~data['운송장_건수'].isnull()]
test = data[data['운송장_건수'].isnull()]

In [ ]:
y_train=train['운송장_건수']
X_train = train.drop('운송장_건수', axis=1)
X_test = test.drop('운송장_건수', axis=1)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 31683
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   송하인_격자공간고유번호       31684 non-null  int64  
 1   수하인_격자공간고유번호       31684 non-null  int64  
 2   물품_카테고리            31684 non-null  int64  
 3   송하인_운송장_건수_MEAN_x  31684 non-null  float64
 4   송하인_운송장_건수_MAX_x   31684 non-null  float64
 5   송하인_운송장_건수_SUM_x   31684 non-null  float64
 6   수하인_운송장_건수_MEAN    31684 non-null  float64
 7   수하인_운송장_건수_MAX     31684 non-null  float64
 8   수하인_운송장_건수_SUM     31684 non-null  float64
 9   물품_운송장_건수_MEAN     31684 non-null  float64
 10  물품_운송장_건수_MAX      31684 non-null  int64  
 11  물품_운송장_건수_SUM      31684 non-null  int64  
 12  송하인_+운송장_건수_MEAN   31684 non-null  float64
 13  송하인_+운송장_건수_MAX    31684 non-null  float64
 14  송하인_+운송장_건수_SUM    31684 non-null  float64
 15  송하인_운송장_건수_MEAN_y  31684 non-null  float64
 16  송하인_운송장_건수_MAX_y   316

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7920 entries, 0 to 7919
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   송하인_격자공간고유번호       7920 non-null   int64  
 1   수하인_격자공간고유번호       7920 non-null   int64  
 2   물품_카테고리            7920 non-null   int64  
 3   송하인_운송장_건수_MEAN_x  7920 non-null   float64
 4   송하인_운송장_건수_MAX_x   7920 non-null   float64
 5   송하인_운송장_건수_SUM_x   7920 non-null   float64
 6   수하인_운송장_건수_MEAN    1973 non-null   float64
 7   수하인_운송장_건수_MAX     1973 non-null   float64
 8   수하인_운송장_건수_SUM     1973 non-null   float64
 9   물품_운송장_건수_MEAN     7920 non-null   float64
 10  물품_운송장_건수_MAX      7920 non-null   int64  
 11  물품_운송장_건수_SUM      7920 non-null   int64  
 12  송하인_+운송장_건수_MEAN   7920 non-null   float64
 13  송하인_+운송장_건수_MAX    7920 non-null   float64
 14  송하인_+운송장_건수_SUM    7920 non-null   float64
 15  송하인_운송장_건수_MEAN_y  7920 non-null   float64
 16  송하인_운송장_건수_MAX_y   7920 

# Modeling

### CatBoost

In [ ]:
cat_features=['송하인_격자공간고유번호','수하인_격자공간고유번호','물품_카테고리','물품_카테고리',
                '송하인_']

In [ ]:
model = CatBoostRegressor(n_estimators=2000, learning_rate=0.02, cat_features=cat_features)
model.fit(X_train, y_train, verbose=False)

### Predict

In [ ]:
predict = model.predict(X_test)
predict

array([5.41451572, 5.08573166, 4.94722053, ..., 4.89384883, 4.89384883,
       4.91270793])

In [ ]:
submission['운송장_건수'] = predict

In [ ]:
submission.to_csv('dacon.csv',index = False)